# kNN vs. SVM

A very common workflow is to index some data based on its embeddings and then given a new query embedding retrieve the most similar examples with k-Nearest Neighbor search. For example, you can imagine embedding a large collection of papers by their abstracts and then given a new paper of interest retrieve the most similar papers to it.

TLDR in my experience it ~always works better to use an SVM instead of kNN, if you can afford the slight computational hit. Example below:

In [1]:
import numpy as np
np.random.seed(42)

embeddings = np.random.randn(1000, 1536) # 1000 documents, 1536-dimensional embeddings
embeddings = embeddings / np.sqrt((embeddings**2).sum(1, keepdims=True)) # L2 normalize the rows, as is common

query = np.random.randn(1536) # the query vector
query = query / np.sqrt((query**2).sum())

In [2]:
# Tired: use kNN
similarities = embeddings.dot(query)
sorted_ix = np.argsort(-similarities)
print("top 10 results:")
for k in sorted_ix[:10]:
  print(f"row {k}, similarity {similarities[k]}")

top 10 results:
row 545, similarity 0.07956628031855817
row 790, similarity 0.0710937236589117
row 973, similarity 0.0692079948121463
row 597, similarity 0.0647482457550396
row 479, similarity 0.06350781255023308
row 229, similarity 0.061432183499702385
row 976, similarity 0.06122285352624162
row 568, similarity 0.06088872280511322
row 800, similarity 0.06007081261453451
row 654, similarity 0.05815882432824042


In [3]:
# Wired: use an SVM
from sklearn import svm

# create the "Dataset"
x = np.concatenate([query[None,...], embeddings]) # x is (1001, 1536) array, with query now as the first row
y = np.zeros(1001)
y[0] = 1 # we have a single positive example, mark it as such

# train our (Exemplar) SVM
# docs: https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
clf = svm.LinearSVC(class_weight='balanced', verbose=False, max_iter=10000, tol=1e-6, C=0.1)
clf.fit(x, y) # train

# infer on whatever data you wish, e.g. the original data
similarities = clf.decision_function(x)
sorted_ix = np.argsort(-similarities)
print("top 10 results:")
for k in sorted_ix[:10]:
  print(f"row {k}, similarity {similarities[k]}")

top 10 results:
row 0, similarity 0.9797112617216354
row 546, similarity -0.8360649738915676
row 791, similarity -0.8519226181122039
row 974, similarity -0.8585435504683989
row 480, similarity -0.8620392370633865
row 598, similarity -0.8653315003700208
row 230, similarity -0.8671983886478063
row 569, similarity -0.8674761579346136
row 977, similarity -0.8705646065664835
row 801, similarity -0.8728033782558366


In practice you will find that this ordering:

- is of higher quality
- is slower: we have to train an SVM
- can easily accommodate a number of positives not just one, so it is more flexible
- don't be scared of having a single positive and everything else being negative. this is totally fine!
- if you have way way too many negatives, consider subsampling and only using a portion of them.

**Value of C**: You'll want to tune C. You'll most likely find the best setting to be between 0.01 and 10. Values like 10 very severely penalize the classifier for any mispredictions on your data. It will make sure to fit your data. Values like 0.01 will incur less penalty and will be more regularized. Usually this is what you want. I find that in practice a value like 0.1 works well if you only have a few examples that you don't trust too much. If you have more examples and they are very noise-free, try more like 1.0

**Why does this work?** In simple terms, because SVM considers the entire cloud of data as it optimizes for the hyperplane that "pulls apart" your positives from negatives. In comparison, the kNN approach doesn't consider the global manifold structure of your entire dataset and "values" every dimension equally. The SVM basically finds the way that your positive example is unique in the dataset, and then only considers its unique qualities when ranking all the other examples.

Ok cool try it out.